<a href="https://colab.research.google.com/github/OmarFares20/OmarFares/blob/main/Cleaned%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!git clone https://github.com/OmarFares20/OmarFares
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 120)

Cloning into 'OmarFares'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), 77.28 KiB | 2.15 MiB/s, done.


In [7]:
!pip install pandas openpyxl


In [9]:
import pandas as pd
import numpy as np
df = pd.read_csv('OmarFares/Nike Dataset.csv')
print("Dataset Info:")
print(df.info())
print("\nFirst 5 rows:")
print(df.head())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9360 entries, 0 to 9359
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Invoice Date    9360 non-null   object
 1   Product         9360 non-null   object
 2   Region          9360 non-null   object
 3   Retailer        9360 non-null   object
 4   Sales Method    9360 non-null   object
 5   State           9360 non-null   object
 6   Price per Unit  9360 non-null   int64 
 7   Total Sales     9360 non-null   int64 
 8   Units Sold      9360 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 658.3+ KB
None

First 5 rows:
  Invoice Date                    Product     Region     Retailer Sales Method     State  Price per Unit  Total Sales  \
0   01/01/2020      Men's Street Footwear  Northeast  Foot Locker     In-store  New York              50         6000   
1   02/01/2020    Men's Athletic Footwear  Northeast  Foot Locker     In-store  Ne

In [10]:
import pandas as pd
df = pd.read_csv("Nike Dataset.csv")

print("=== Initial Data Info ===")
print(df.info())
print("\n=== First few rows ===")
print(df.head())
print("\n=== Data Shape ===")
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

initial_rows = df.shape[0]
df = df.drop_duplicates()
duplicates_removed = initial_rows - df.shape[0]
print(f"\nRemoved {duplicates_removed} duplicate rows")

print("\n=== Missing Values Summary ===")
print(df.isnull().sum())

df["Invoice Date"] = pd.to_datetime(df["Invoice Date"], format="%d/%m/%Y", errors="coerce")
print(f"\nDate conversion - Number of invalid dates: {df['Invoice Date'].isnull().sum()}")

numeric_cols = ["Price per Unit", "Total Sales", "Units Sold"]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

print("\n=== Negative Values Check ===")
for col in numeric_cols:
    neg_count = (df[col] < 0).sum()
    print(f"{col}: {neg_count} negative values")
    if neg_count > 0:

        df = df[df[col] >= 0]

print("\n=== Data Quality Checks ===")

df["Calculated Units"] = (df["Total Sales"] / df["Price per Unit"]).round(0)
mismatch = (df["Units Sold"] != df["Calculated Units"]).sum()
print(f"Units Sold mismatch with calculation: {mismatch} rows")

df = df.drop(columns=["Calculated Units"])

price_stats = df["Price per Unit"].describe()
print(f"\nPrice per Unit stats:\nMin: ${price_stats['min']:.2f}, Max: ${price_stats['max']:.2f}, Mean: ${price_stats['mean']:.2f}")

categorical_cols = ["Product", "Region", "Retailer", "Sales Method", "State"]
for col in categorical_cols:
    df[col] = df[col].str.strip().str.title()
    print(f"{col}: {df[col].nunique()} unique values")

units_stats = df["Units Sold"].describe()
Q1 = units_stats["25%"]
Q3 = units_stats["75%"]
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df["Units Sold"] < lower_bound) | (df["Units Sold"] > upper_bound)]
print(f"\nUnits Sold outliers (IQR method): {len(outliers)} rows")

df["Year"] = df["Invoice Date"].dt.year
df["Month"] = df["Invoice Date"].dt.month
df["Quarter"] = df["Invoice Date"].dt.quarter
df["Day_of_Week"] = df["Invoice Date"].dt.day_name()

df["Sales_Check_Flag"] = abs(df["Total Sales"] - (df["Price per Unit"] * df["Units Sold"])) > 0.01

print("\n=== Final Data Quality Summary ===")
print(f"Total rows after cleaning: {df.shape[0]}")
print(f"Total columns after cleaning: {df.shape[1]}")
print(f"Date range: {df['Invoice Date'].min()} to {df['Invoice Date'].max()}")
print(f"Total Sales sum: ${df['Total Sales'].sum():,.2f}")
print(f"Total Units Sold: {df['Units Sold'].sum():,}")

print("\n=== Summary by Region ===")
region_summary = df.groupby("Region").agg({
    "Total Sales": "sum",
    "Units Sold": "sum",
    "Invoice Date": "nunique"
}).round(2)
region_summary.columns = ["Total_Sales", "Total_Units", "Transaction_Days"]
print(region_summary)

print("\n=== Top 5 Retailers by Sales ===")
retailer_summary = df.groupby("Retailer").agg({
    "Total Sales": "sum",
    "Units Sold": "sum"
}).round(2).sort_values("Total Sales", ascending=False).head()
print(retailer_summary)

df.to_csv("Cleaned_Nike_Dataset.csv", index=False)
print("\n=== Cleaning Complete ===")
print(f"Cleaned data saved to 'Cleaned_Nike_Dataset.csv'")


=== Initial Data Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9360 entries, 0 to 9359
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Invoice Date    9360 non-null   object
 1   Product         9360 non-null   object
 2   Region          9360 non-null   object
 3   Retailer        9360 non-null   object
 4   Sales Method    9360 non-null   object
 5   State           9360 non-null   object
 6   Price per Unit  9360 non-null   int64 
 7   Total Sales     9360 non-null   int64 
 8   Units Sold      9360 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 658.3+ KB
None

=== First few rows ===
  Invoice Date                    Product     Region     Retailer Sales Method     State  Price per Unit  Total Sales  \
0   01/01/2020      Men's Street Footwear  Northeast  Foot Locker     In-store  New York              50         6000   
1   02/01/2020    Men's Athletic Footwear  Northeast  Foot Lo